# Compile the whole code and creating a Pipeline

In [5]:
import numpy as np
from scipy import signal
from scipy.interpolate import interp1d
from scipy.signal import butter, filtfilt
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import h5py as h5

### All steps in methods

In [2]:
fs = 4096

def whiten(strain, dt):
    NFFT = 1*fs
    fmin = 10
    fmax = 2000 
    Pxx, freqs = mlab.psd(strain, Fs = fs, NFFT = NFFT)
    psd = interp1d(freqs, Pxx)
    
    Nt = len(strain)   
    freqs_w = np.fft.rfftfreq(Nt, dt)
    
    hf = np.fft.rfft(strain)
    white_hf = hf / (np.sqrt(psd(freqs_w) /dt/2.))
    white_ht = np.fft.irfft(white_hf, n=Nt)    

    return white_ht

def bandpassing(strain_whiten):
    bb, ab = butter(4, [20.*2./fs, 300.*2./fs], btype='band')
    strain_whitenbp = filtfilt(bb, ab, strain_whiten)
    
    return strain_whitenbp

def define_time(h5_file):
    Xstart_time = h5_file['strain']['Strain'].attrs['Xstart']
    Xend_time = Xstart_time + 32
    Xspacing_time =  h5_file['strain']['Strain'].attrs['Xspacing']

    time = np.arange(Xstart_time, Xend_time, Xspacing_time)
    return time
    
def get_strain(h5_file):
    return h5_file['strain']['Strain']

def save_filtered_data(h5_file, data):
    h5_file['strain'].create_dataset('Strain_whitenbp', data=data)

### Apply the steps on the data

In [4]:
def main():
    h5_files_name = ['L-L1_GWOSC_4KHZ_R1-1126259447-32.hdf5', 'H-H1_GWOSC_4KHZ_R1-1126259447-32.hdf5']

    h5_files = [h5.File(x, 'r') for x in h5_files_name]
    
    time = define_time(h5_files[0])

    # Define the value of dt
    dt = time[1] - time[0]

    # Create a pipeline
    strain = get_strain(h5_files[0])
    strain = whiten(strain, dt)
    strain = bandpassing(strain)
     
    print("Finish transform.")

In [5]:
main()

Finish transform.


# Store a Pandas DataFrame into a hdf5 file

We can store different types of data into our HDF5 file. And it is possible to use the Pandas lib, for the Pandas lovers, to do it.

We will create a small dataframe and use the Pandas library.

In [2]:
import pandas as pd
data = {
        'GW': ['GW150914', 'GW170104', 'GW190521'],
        'm1 (solar mass)': [36., 31.2, 85.],
        'm2 (solar mass)': [29., 19.4, 66.],
        'm_final (solar mass)': [62., 48.7, 142.]
    }
df = pd.DataFrame(data)

Now it is possible to use the pandas lib and save the file.

In [27]:
df.to_hdf("pandas_example.h5", key="/collisionData/d1", data_columns=True, format='table');

In [39]:
collision.close()
collision = h5.File('pandas_example.h5', 'a')
collision.keys()

<KeysViewHDF5 ['collisionData']>

In [29]:
list(collision['collisionData']['d1'].items())

[('_i_table', <HDF5 group "/collisionData/d1/_i_table" (5 members)>),
 ('table', <HDF5 dataset "table": shape (3,), type "|V40">)]

In [34]:
collision['collisionData']['d1']['table'][()]

array([(0, b'GW150914', 36. , 29. ,  62. ),
       (1, b'GW170104', 31.2, 19.4,  48.7),
       (2, b'GW190521', 85. , 66. , 142. )],
      dtype=[('index', '<i8'), ('GW', 'S8'), ('m1 (solar mass)', '<f8'), ('m2 (solar mass)', '<f8'), ('m_final (solar mass)', '<f8')])

### Create a attibute

In [40]:
collision['collisionData']['d1'].attrs['name'] = 'Name'

In [41]:
collision['collisionData']['d1'].attrs['name']

'Name'

In [ ]:
collision['collisionData'].create_dataset('panda', data=data.values())

TypeError: Object dtype dtype('O') has no native HDF5 equivalent